In [ ]:
#====================================================================================================
# loRA 파인튜닝 예제2
# - quantumaikr/KoreanLM 모델을 이용하여 PEFT(Parameter-Efficient Fine-tuning)->LoRA(Low-Rank Adaptation) 기법으로 파인튜닝하는 예시
#
# 참조 DOC: https://github.com/quantumaikr/KoreanLM/tree/main
# 참고소스: https://github.com/quantumaikr/KoreanLM/blob/main/finetune-lora.py
# 참고소스 : https://github.com/quantumaikr/KoreanLM/blob/main/generate.py
#
# <package 설치> 
# peft: pip install peft
# load_in_8bit : pip install -i https://test.pypi.org/simple/ bitsandbytes-cudaXXX  (XXX는 CUDA version (e.g. 11.6 = 116))
# transfomers 4.27.1 이상으로 업데이트 : pip install -U transformers[pytorch]
# dispatch_model() got an unexpected keyword argument 'offload_index' 오류 => accelerate 업데이트 : pip install -U accelerate
# module 'bitsandbytes.nn' has no attribute 'Linear8bitLt' => pip install bitsandbytes==0.37.2
# 'MatmulLtState' object has no attribute 'memory_efficient_backward' 오류 => bitsandbytes 버전 0.37.2 설치 : pip install bitsandbytes==0.37.2
#====================================================================================================
import os
import sys
import time
import torch
import transformers
from tqdm.notebook import tqdm

from myutils import GPU_info, seed_everything, mlogging, SaveBERTModel, AccuracyForMLM

# wand 비활성화 
# => trainer 로 훈련시키면 기본이 wandb 활성화이므로, 비활성화 시킴
os.environ["WANDB_DISABLED"] = "true"

# 입력 모델 경로
model_path:str ='../data11/model/LLM/beomi/KoAlpaca-Polyglot-5.8B'
#model_path='beomi/KoRWKV-1.5B'

# 훈련 관련 출력 경로
out_dir:str = '../data11/model/LLM/out/'

device = GPU_info()
print(device)

#seed 설정
SEED = 111
seed_everything(SEED)

#logging 설정
logger =  mlogging(loggername="beomi-ft", logfilename="../log/beomi-ft")

In [ ]:
'''
# KorQuAD_v1.0_train.json 혹은 KorQuAD_v1.0_dev.json 파일을 불러와서, sLLM 훈련을 위한 json 파일로 만듬.
from dataclasses import dataclass
import json
from tqdm.notebook import tqdm

@dataclass
class QAExample:
    # 질문 : 임종석이 여의도 농민 폭력 시위를 주도한 혐의로 지명수배 된 날은?
    question_text: str
    # (답 찾는 대상인)지문 : 1989년 2월 15일 여의도 농민 폭력 시위를 주도한 혐의 ... 서울지방경찰청 공안분실로 인계되었다.
    context_text: str
    # 답변 : 1989년 2월 15일
    answer_text: str
    
corpus_fpath = '../data11/korpora/korQuAD/KorQuAD_v1.0_dev.json'
examples = []
         
# KorQuAD_v1.0_train.json 파일을 불러옴
json_data = json.load(open(corpus_fpath, "r", encoding="utf-8"))["data"]
for entry in tqdm(json_data):
        for paragraph in entry["paragraphs"]:
            context_text = paragraph["context"]
            for qa in paragraph["qas"]:
                question_text = qa["question"]
                for answer in qa["answers"]:
                    answer_text = answer["text"]
                    start_position_character = answer["answer_start"]

                    # question, context, answer, startposition 등을 설정함
                    if question_text and answer_text and context_text and start_position_character:
                        doc = {}
                        doc['question'] = question_text
                        doc['context'] = context_text
                        doc['answer'] = answer_text
                        
                        examples.append(doc)
                        
docs ={}
docs['text'] = examples

#json 파일로 저장.
# JSON 파일을 엽니다.
with open("KorQuAD_v1.0_dev_data.json", "w") as f:
    # 리스트를 JSON으로 변환합니다.
    json.dump(docs, f, indent=4)

# JSON 파일을 닫습니다.
f.close()

# load_data로 불러와서 테스트 해봄.
from datasets import load_dataset 
data = load_dataset("json", data_files='KorQuAD_v1.0_dev_data.json', field="text")

print(data)
print(data['train'][0])
'''

In [ ]:
# CausalLM 모델 로딩
model = transformers.AutoModelForCausalLM.from_pretrained(
        model_path,
        load_in_8bit=True,           # 8bit 양자화.
        torch_dtype=torch.float16,
        device_map="auto",
        #cache_dir="./cache/",
    )

# tokenizer 로딩
tokenizer = transformers.AutoTokenizer.from_pretrained(
        model_path,
        padding_side="right",
        use_fast=False,
        #cache_dir="./cache/",
    )

if tokenizer.pad_token is None:
    tokenizer.add_special_tokens({'pad_token': '[PAD]'})
    
print(model)

In [ ]:
# LoRA 모델 설정 
from typing import List

from peft import (
    LoraConfig,
    get_peft_model,
    get_peft_model_state_dict,
    prepare_model_for_int8_training,
    set_peft_model_state_dict,
)

# int8 양자화 처리를 위해, 전처리 함.
model = prepare_model_for_int8_training(model)

# **Polyglot 모델인 경우에만 주석처리
'''
# loRA config 설정
lora_target_modules: List[str] = [
        "q_proj",
        "v_proj",
    ]
'''
config = LoraConfig(
        r=8,
        lora_alpha=16,
        #target_modules=lora_target_modules,
        lora_dropout=0.05,
        bias="none",
        task_type="CAUSAL_LM",
)

# CausalLM 모델과 loRA 모델 연동
model = get_peft_model(model, config)

In [ ]:
# LoRA 훈련할 파라메터 계수 출력 해봄
model.print_trainable_parameters()

In [ ]:
# 말뭉치 로딩
import os.path as osp
import json
from datasets import load_dataset

# param---------------------------------------------------
data_path='./data/moco_qa_summerize_gpt.json'  # 말뭉치 파일경로
val_set_size = 75                # 평가 말뭉치 사이즈   
cutoff_len=1024                   # 토크너나이저 길이
add_eos_token=True               # True=말뭉치 뒤에 add_eos_token(</s>) 추가함 
shuffle = True                   # True=말뭉치를 랜덤하게 섞은다.

# prompt_template(모델에 따라 변경)
PROMPT_DICT = {    
    #"prompt_context":("###지시: [질의]에 가장 적합한 [응답]을, 제시된 [문단]에서 찾아서 간단한 문장으로 답변해주세요.\n\n###[질의]: {query}\n\nn###[문단]: {context}\n\n###[응답]:"),
    #"prompt_no_context":("###지시: 질의에 대해, 간단한 문장으로 응답해 주세요 ###[질의]: {query}\n\n###[응답]:")
    
    #"prompt_context":("###지시: 문단에서 질의에 대해 가장 적합한 내용을 찾아, 응답 문장을 만들어 주세요.\n\n###[문단]: {context}\n\n###[질의]: {query}\n\n###[응답]:"),
    #"prompt_no_context":("###지시: 질의에 대해, 자세하게 응답 문장을 만들어 주세요.\n\n###[질의]: {query}\n\n###[응답]:")
    
    "prompt_context":("###지시: 아래 문단 내용을 요약해서, 질의에 맞게 응답 문장을 만들어 주세요.\n\n###[문단]: {context}\n\n###[질의]: {query}\n\n###[응답]:"),
    "prompt_no_context":("###지시: 질의에 대해, 자세하게 응답 문장을 만들어 주세요.\n\n###[질의]: {query}\n\n###[응답]:")
}

# prompt 테스트 
print(f'\n\nprompt 테스트----------------------\n')
query='제주도의 크기는 얼마?'
#context = ''
context = '제주도는 최남단에 있는 섬으로, 길이는 40km가 되는 대한민국에서 가장큰 섬이다.'
if context:
    prompt = PROMPT_DICT['prompt_context'].format(query=query, context=context)
else:
    prompt = PROMPT_DICT['prompt_no_context'].format(query=query)
    
print(prompt)

#--------------------------------------------------------

# 실제 입력 말뭉치를 tokenizer 하고, label고 만드는 과정
def tokenize(prompt, add_eos_token=True):
    # there's probably a way to do this with the tokenizer settings
    # but again, gotta move fast
    result = tokenizer(
        prompt,
        truncation=True,
        max_length=cutoff_len,
        padding=False,
        return_tensors=None,
    )
    
    # eos 토큰을 추가함.
    if (
        result["input_ids"][-1] != tokenizer.eos_token_id
        and len(result["input_ids"]) < cutoff_len
        and add_eos_token
    ):
        result["input_ids"].append(tokenizer.eos_token_id)
        result["attention_mask"].append(1)

    # label은 input_ids 복사해서 만듬.
    result["labels"] = result["input_ids"].copy()

    return result

#---------------------------------------------------
# 말뭉치 종류에 땨라, 훈련 데이터 생성
# - 말뭉치 종류에 따라 프롬프트 템블릿에 맞추어 수정해야 함.
#---------------------------------------------------

def generate_and_tokenize_prompt(data_point):

    # templeate 로딩함
    #description:"KoreanLM-LoRA에서 사용하는 템플릿입니다."
    #prompt_input:"아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요. ### Instruction: {instruction} ### Input: {input} ### Response: "
    #prompt_no_input:"아래는 작업을 설명하는 명령어입니다. 요청을 적절히 완료하는 응답을 작성하세요. ### Instruction: {instruction} ### Response: "
    #response_split:"### Response:"
    '''
    file_name = "./data/templates/korean.json"
    #file_name = "./data/templates/qanda.json" # 말뭉치에 맞게 템플릿을 변경해야함
    if not osp.exists(file_name):
        raise ValueError(f"Can't read {file_name}")
            
    with open(file_name) as fp:
        PROMPT_DICT = json.load(fp)
    '''
    '''
    # input 있으면 instruction+input 시킴
    if data_point["input"]:
        res = PROMPT_DICT["prompt_context"].format(query=data_point["instruction"], context=data_point["input"])
    else:
        res = PROMPT_DICT["prompt_no_context"].format(query=data_point["instruction"])    
    

     # output있으면 +output 시켜줌.
    if data_point["output"]:
        full_prompt = f'{res}{data_point["output"]}'
    else:
        full_prompt = res
    '''
  
    # context 있으면 instruction+input 시킴
    if data_point["context"]:
        res = PROMPT_DICT["prompt_context"].format(query=data_point["question"], context=data_point["context"])
    else:
        res = PROMPT_DICT["prompt_no_context"].format(query=data_point["question"])    
   
     # output있으면 +output 시켜줌.
    if data_point["answer"]:
        full_prompt = f'{res}{data_point["answer"]}'
    else:
        full_prompt = res
     
    # tokenizer 처리
    tokenized_full_prompt = tokenize(full_prompt)
    
    return tokenized_full_prompt
#---------------------------------------------------


# 말뭉치 로딩
if data_path.endswith(".json") or data_path.endswith(".jsonl"):
    #----------------------------------------------------------
    # {
    #     text[
    #         {},
    #         {},
    #         {}
    #     ]
    # } 
    # 식으로 json 파일 구조가 되어 있는 파일이어야 함.
    #----------------------------------------------------------
    data = load_dataset("json", data_files=data_path, field="text")
else:
    data = load_dataset(data_path)     
    

# train, test 말뭉치로 분할
if val_set_size > 0:
    train_val = data["train"].train_test_split(
        test_size=val_set_size, shuffle=shuffle, seed=SEED
    )
    train_data = (
        train_val["train"].shuffle().map(generate_and_tokenize_prompt)
    )
    val_data = (
        train_val["test"].shuffle().map(generate_and_tokenize_prompt)
    )
else:
    train_data = data["train"].shuffle().map(generate_and_tokenize_prompt)
    val_data = None

In [ ]:
print(train_data[1])
print()
print('len:')
print(len(train_data[1]['input_ids']))
print(tokenizer.decode(train_data[1]['input_ids']))

In [ ]:
# 모델 훈련 param 설정
micro_batch_size:int = 4
batch_size:int = 128
gradient_accumulation_steps:int = int(batch_size // micro_batch_size)
num_epochs:int = 20
learning_rate: float = 3e-4

trainer = transformers.Trainer(
        model=model,
        train_dataset=train_data,
        eval_dataset=val_data,
        args=transformers.TrainingArguments(
            per_device_train_batch_size=micro_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            warmup_steps=100,
            num_train_epochs=num_epochs,
            learning_rate=learning_rate,
            fp16=True,
            logging_steps=10,
            optim="adamw_torch",
            evaluation_strategy="epoch" if val_set_size > 0 else "no",
            save_strategy="epoch",
            #eval_steps=200 if val_set_size > 0 else None,
            #save_steps=200,
            output_dir=out_dir,
            save_total_limit=2,
            #load_best_model_at_end=True if val_set_size > 0 else False,
            #ddp_find_unused_parameters=False if ddp else None,
            group_by_length=False,
            #report_to="wandb" if use_wandb else None,
            #run_name=wandb_run_name if use_wandb else None,
        ),
    
        # DataCollatorForSeq2Seq 로 지정.
        data_collator=transformers.DataCollatorForSeq2Seq(
            tokenizer, pad_to_multiple_of=8, return_tensors="pt", padding=True
        ),
)

In [ ]:
# 훈련 시작
model.config.use_cache = False

old_state_dict = model.state_dict

model.state_dict = (
    lambda self, *_, **__: get_peft_model_state_dict(
        self, old_state_dict()
    )
).__get__(model, type(model))


#trainer.train(resume_from_checkpoint=resume_from_checkpoint)
trainer.train()

In [ ]:
# 훈련 완료후 저장
import os

OUT_PATH = '../data11/model/LLM/beomi/KoAlpaca-Polyglot-5.8B-LoRA-qa-moco-2/'
os.makedirs(OUT_PATH, exist_ok=True) # 폴더 생성

# adapter_config.json 와 adapter_config.bin 파일 생성. 
model.save_pretrained(OUT_PATH)
 
# 실제 모델 파일 (pytorch_model.bin, training_args.bin)
trainer.save_state()  # 모델이 모든 가중치, 학습률, 알고리즘등을 저장.
trainer.save_model(output_dir=OUT_PATH) # 모델이 가중치만 저장

# pytorch_model.bin 이름을 adapter_model.bin 변경.
# => 그래야 huggingface 를 이용해서 불러올수 있음.
if os.path.exists(os.path.join(OUT_PATH, 'adapter_model.bin')):
    os.remove(os.path.join(OUT_PATH, 'adapter_model.bin'))
    print(f'remove adapter_model.bin')

if os.path.exists(os.path.join(OUT_PATH, 'pytorch_model.bin')):
    os.rename(os.path.join(OUT_PATH, 'pytorch_model.bin'), os.path.join(OUT_PATH, 'adapter_model.bin'))
    print(f'rename pytorch_model.bin')


In [ ]:
# 여기서부터는 저장된 모델 평가하는 코드임.
# 참고 소스 : https://github.com/quantumaikr/KoreanLM/blob/main/generate.py
#
# 1. prompt는 모델마다 훈련 프롴프트가 다르므로, 해당 모델 prompt 로 입력하는게 보나 나은 출력이 나옴.
# => KoAlpaca, KoAlpaca-Polyglot-5.8B prompt 예시=> '### 질문: {input_text}\n\n### 맥락: {context}\n\n### 답변:'
#
# => open_llama_7b, KoreanLM, KoreanLM-hf prompt 예시(data/korean.json 파일 참조)
#  "아래는 작업을 설명하는 명령어와 추가 컨텍스트를 제공하는 입력이 짝을 이루는 예제입니다. 요청을 적절히 완료하는 응답을 작성하세요. ### Instruction: {instruction} ### Input: {input} ### Response: "

In [ ]:
# 여기서부터는 저장된 모델 평가하는 코드임.

In [ ]:
import torch
import transformers
from peft import PeftModel
import time

from myutils import GPU_info
device = GPU_info()
    
#====================================================================
# param 설정
lora_weights:str = '../data11/model/LLM/beomin/KoAlpaca-Polyglot-5.8B/'
model_path:str ='../data11/model/LLM/beomi/KoAlpaca-Polyglot-5.8B/'

uselora_weight = False # Lora 사용하는 경우 True
load_8bit = False
usekoalphaca = True   # Koalpaca 모델인 경우엔 True 해줘야함.
#====================================================================

start_time = time.time()

# tokenizer 로딩
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_path,
    padding_side="right",
    use_fast=False,
)

# 원본 모델 로딩
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_path,
    load_in_8bit=load_8bit,
    torch_dtype=torch.float16,
    device_map="auto",
)


if uselora_weight:
    # loRA 모델 로딩
    model = PeftModel.from_pretrained(
            model,
            lora_weights,
            torch_dtype=torch.float16,
    )


if not load_8bit:
    model.half()
    
model.eval()

end_time = time.time() - start_time
print("time: {:.2f} ms\n".format(end_time * 1000)) 


In [ ]:
import os.path as osp
from typing import Union
import json
from transformers import GenerationConfig
#print(device)

# open_llama_7b, KoreanLM, KoreanLM-hf prompt 템플릿 경로
template_file_name = "./data/templates/korean.json"

# open_llama_7b, KoreanLM, KoreanLM-hf prompt 만드는 함수.
def generate_prompt(
        instruction: str,               # 설명
        input: Union[None, str] = None, # 입력:옵션
        label: Union[None, str] = None,
    ) -> str:
    
    
    if not osp.exists(template_file_name):
        raise ValueError(f"Can't read {template_file_name}")
            
    with open(template_file_name) as fp:
        template = json.load(fp)
        
    # returns the full prompt from instruction and optional input
    # if a label (=response, =output) is provided, it's also appended.
    if input:
        res = template["prompt_input"].format(instruction=instruction, input=input)
    else:
        res = template["prompt_no_input"].format(instruction=instruction)
        
    if label:
        res = f"{res}{label}"
    
    return res
   
'''
대한민국의 남서쪽에 있는 섬으로 행정구역 상 광역자치단체인 제주특별자치도에 속한다. 
한국의 섬 중에서 가장 크고 인구가 많은 섬이기도 하며 면적은 1,833.2㎢이다. 
이는 대한민국 본토에서 가장 큰 기초자치단체인 홍천군(1,820.14㎢)보다 약간 크며, 제주도 다음 두 번째로 큰 섬인 거제도(379.5㎢)의 5배 정도 된다. 
인구는 약 70만 명, 세계 섬 크기 218위이다.
제주도는 동아시아권 전체로 범위를 넓혀도 꽤 큰 섬에 속한다. 
6,000개가 넘는 섬이 있는 일본조차도 본토로 간주되는 혼슈, 홋카이도, 시코쿠, 규슈 4개 섬을 제외한 나머지 모든 섬이 제주도보다 작다.
중국도 하이난 섬 한 곳만이 제주도보다 클 뿐이다.
하와이에서도 최대 섬인 하와이 섬 다음으로 큰 섬인 마우이 섬이 제주도보다 약간 큰 정도이다. 
미국도 본토만 따지면 제주도보다 큰 섬은 롱아일랜드 뿐이다. 
프랑스도 본토에는 제주도보다 큰 섬이 코르시카 섬밖에 없고, 독일에서 가장 큰 섬인 뤼겐 섬은 제주도보다 작다. 
크기에 대한 직접적인 비교를 하자면 제주도의 동서 길이 약 73km, 
남 길이 약 31km를 대입하여 서울시청 기준 동서 길이로 인천광역시 서구 오류동 거첨도에서 출발하여 
경기도 양평군 서종면에 도달하고 남북 길이로는 송추계곡에서 출발하여 관악산에 이르는 수준이다.

질문 : 제주도 길이는 얼마?
'''

instruction ='''
아래 내용을 가지고 질문에 간단히 답변해 주세요

학자금 지원
목적 : 임직원 자녀의 학자금을 지급함으로써 임직원의 복지 향상, 근로의욕의 제고 및 장기근속 유도
지원 기준
임직원의 고등학교 재학중인 자녀 학자금 지원
고등학교 교육비(입학금, 수업료, 학교운영지원금, 교과서)
자녀수 제한은 없으며, 자녀당 연간 300만원 한도에서 지원
지급신청 : 학자금의 신청은 아래 신청시기 해당월 이내에 다음 각호의 신청서류를 구비하여 해당부서장 결재 득 후 경영지원본부에 신청
학자금 신청서 (별첨1)
납입고지서 영수증 사본
취학자녀의 재학증명서(최초 신청시 1회에 한함)
가족관계증명원(최초 신청시 1회에 한함)
신청시기 : 납부 후 1개월 이내
지급제외 대상
휴학, 퇴학 등의 사유로 학적이 변동된 경우
타기관, 단체, 학교에서 장학금을 받는 경우 (일부를 받는 경우는 차액 지급)
취학자녀의 부모가 임직원으로 동시에 지급대상일 경우 1인만 적용
기타 : 해당금액은 과세대상임
'''

#instruction = "엠파워 이지스-씨 제품 설명"

input_text = '''
학자금 신청 기준은 어떻게 되나?
'''
# prompt 구성
#========================================================================
# koAlphaca일때 prompt 구성.
# => "### 질문: {input}\n\n### 맥락: {context}\n\n### 답변:" 식으로 입력. 
# => '### 맥락' 은 이전 대화 맥락임.
#=========================================================================

if usekoalphaca:
    input_text = instruction+'\r\n'+input_text
    context = "" #이전 대화 맥락 입력
    
    prompt = f"### 질문: {input_text}\n\n### 맥락: {context}\n\n### 답변:" if context else f"### 질문: {input_text}\n\n### 답변:"
else:
    prompt = generate_prompt(instruction=instruction, input=input_text)

max_new_tokens = 256

start_time = time.time()

# config 설정
generation_config = GenerationConfig(
            temperature=0.5,
            top_p=0.75,
            top_k=40,
            num_beams=1,
            eos_token_id=tokenizer.eos_token_id,
            pad_token_id=tokenizer.pad_token_id
)

# 프롬프트 tokenizer 
inputs = tokenizer(prompt, return_tensors="pt").to(device)
input_ids = inputs["input_ids"]
#print(input_ids)

# Without streaming
# generate 처리
with torch.no_grad():
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=False,
        max_new_tokens=max_new_tokens,
    )
    
# 출력
#print(generation_output)
#print()

s = generation_output.sequences[0]
output = tokenizer.decode(s)

end_time = time.time() - start_time
print("time: {:.2f} ms\n".format(end_time * 1000)) 
print(output)


In [ ]:
# 원본 모델 평가 해봄